In [ ]:
import pandas as pd
import numpy as np
from openpyxl.utils import get_column_letter

file_in    = "modified_CBEAM_axial_stress_and_2Delements.xlsx"
# sheet_name = "6_redesign_2D"
sheet_name = "5_redesign_CBEAM"
# BLOCK      = 6                          # average every 6 rows 2D elements
BLOCK      = 3                          # average every 3 rows stringer elements
# cols_src   = ["XX","XY","YY"]         # 2d elements average
cols_src   = ["axial_stress"]         # stringer elements average

# 1) read
df = pd.read_excel(file_in, sheet_name=sheet_name, engine="openpyxl")

# 2) make sure the source cols are numeric
df[cols_src] = df[cols_src].apply(pd.to_numeric, errors="coerce")

# 4) build a block id (0,0,0,0,0,0,1,1,1…)
df["grp"] = df.index // BLOCK

# 5) compute block means (same length as df) → new DataFrame
means = (
    df.groupby("grp")[cols_src]
      .transform("mean")                        # N rows × 3 cols
)

# 6) attach the new columns
first_blank = len(df.columns)                     # remember first blank slot

# df[['sig_xx_avg','sig_xy_avg','sig_yy_avg']] = means.values
df[['sig_str_avg']] = means.values

# 7) write those three columns back (overlay keeps sheet intact)
with pd.ExcelWriter(file_in,
                    engine="openpyxl",
                    mode="a",
                    if_sheet_exists="overlay") as w:
    # df[['sig_xx_avg','sig_xy_avg','sig_yy_avg']].to_excel( #2d elements average values
    df[['sig_str_avg']].to_excel( # stringer elements average values
        w,
        sheet_name=sheet_name,
        index=False,
        startcol=first_blank                      # zero-based first free col
    )

col_start = get_column_letter(first_blank + 1)
col_end   = get_column_letter(first_blank + len(cols_src))
print(f"✓ Averages written to {col_start}:{col_end} in '{sheet_name}'")


✓ Averages written to H:J in '6_redesign_2D'
